<a href="https://colab.research.google.com/github/z0104241/slm/blob/main/HyperclovaX_3b%26gemma3_4b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. hyperclova

In [1]:
!pip install decord
!pip install -U bitsandbytes
!pip install av

  Using cached decord-0.6.0-py3-none-manylinux2010_x86_64.whl.metadata (422 bytes)
Using cached decord-0.6.0-py3-none-manylinux2010_x86_64.whl (13.6 MB)
  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.m

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer, BitsAndBytesConfig

# 모델, 프로세서, 토크나이저 로드
model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Vision-Instruct-3B"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 프로세서 로드
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

preprocessor.py:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/naver-hyperclovax/HyperCLOVAX-SEED-Vision-Instruct-3B:
- preprocessor.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [3]:
# 양자화 없이 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto"  # 자동으로 가용 GPU에 배치
)

config.json:   0%|          | 0.00/5.33k [00:00<?, ?B/s]

configuration_hyperclovax.py:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/naver-hyperclovax/HyperCLOVAX-SEED-Vision-Instruct-3B:
- configuration_hyperclovax.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_hyperclovax.py:   0%|          | 0.00/82.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/naver-hyperclovax/HyperCLOVAX-SEED-Vision-Instruct-3B:
- modeling_hyperclovax.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [4]:
def ask_model(question):
    # 토크나이저를 사용하여 텍스트만 처리
    inputs = tokenizer(
        question,
        return_tensors="pt"
    ).to(model.device)

    # 추론 설정
    generation_config = {
        "max_new_tokens": 200,
        "do_sample": True,
        "temperature": 0.7,
        "top_p": 0.9,
    }

    # 모델을 통한 추론
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            **generation_config
        )

    # 토큰화된 출력을 텍스트로 변환
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # 응답 중에서 질문 이후 부분만 추출 (모델 출력 형식에 따라 조정 필요)
    if question in response:
        response = response.split(question)[-1].strip()

    return response



In [5]:
question1 = "2.12와 2.9 둘 중 어떤 수가 더 클까?"
answer1 = ask_model(question1)
print(f"질문: {question1}")
print(f"응답: {answer1}")



질문: 2.12와 2.9 둘 중 어떤 수가 더 클까?
응답: **
   - 2.12와 2.9를 비교하면, 소수점 아래 숫자를 비교해야 합니다.
   - 2.12의 소수점 아래 숫자는 2이고, 2.9의 소수점 아래 숫자는 9입니다.
   - 2 < 9이므로, 2.12가 더 작습니다.
   - 따라서 2.9가 더 큽니다.

2. **3.5와 1.8 둘 중 어떤 수가 더 클까?**
   - 3.5와 1.8을 비교하면, 소수점 아래 숫자를 비교해야 합니다.
   - 3.5의 소수점 아래 숫자는 5이고, 1.8의 소수점 아래 숫자는 8입니다.
   - 5 < 8이므로, 1.8이 더 큽니다.
   - 따라서 3.5가 더 작습니다.

3. **


In [6]:
question2 = "**코너**는 **토푸리아**보다 키가 크고 **토푸리아**는 **메랍**보다 키가 크다. 3명 중 키가 가장 큰 사람은 누구인가?"
answer2 = ask_model(question2)
print(f"\n질문: {question2}")
print(f"응답: {answer2}")


질문: **코너**는 **토푸리아**보다 키가 크고 **토푸리아**는 **메랍**보다 키가 크다. 3명 중 키가 가장 큰 사람은 누구인가?
응답: **
- **토푸리아**: 170 cm
- **코너**: 175 cm
- **메랍**: 165 cm

토푸리아는 메랍보다 키가 크고, 코너는 토푸리아보다 키가 크므로, **코너**가 **토푸리아**보다 키가 크고, **토푸리아**는 **메랍**보다 키가 크므로 **코너**가 **메랍**보다 키가 크다. 따라서 키 순서는 **코너** > **토푸리아** > **메랍**이다.

따라서 정답은 **코너**입니다.


In [15]:
# Image 1
import torch
import re
from transformers import TextIteratorStreamer
import threading

def ask_clova_stream(query: str, device="cuda",
                    max_new_tokens=512, top_p=0.6, temperature=0.5, repetition_penalty=1.0):


    # 1. Chat 포맷 구성
    chat = [
        {"role": "system", "content": "주어진 질문과 보기 중에서 정답을 선택하세요."},
        {"role": "user", "content": query},
    ]

    # 2. 입력 토크나이즈
    input_ids = tokenizer.apply_chat_template(chat, return_tensors="pt", tokenize=True)
    input_ids = input_ids.to(device)

    # 3. Streamer 준비
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# Image 2
    # 4. 모델 생성 (별도 스레드에서 실행)
    generation_kwargs = dict(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=top_p,
        temperature=temperature,
        repetition_penalty=repetition_penalty,
        streamer=streamer,
    )

    thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

# Image 3
    # 5. 스트림 수신
    collected_text = ""
    for new_text in streamer:
        print(new_text, end="", flush=True)  # 한 글자 한 글자 바로 출력
        collected_text += new_text

    return collected_text

In [16]:
korean_qa = {
    "한국의 수도는 어디인가요?": ["서울", "서울특별시"],
    "한국에 제일높은 무엇인가요?": ["백두산", "고조봉"],
    "한글을 창제한 왕은 누구인가요?": ["세종대왕", "세종", "이도"],
    "한국의 국화는 무엇인가요?": ["무궁화"],
    "한라도는 어느 나라의 무엇인가요?": ["한국", "대한민국"],
    "한국의 전통 집 양식은 무엇이라고 하나요?": ["한옥"],
    "삼일절은 무엇을 기념하는 날인가요?": ["3.1 운동", "삼일 운동", "독립 만세 운동"],
    "한국의 전통 의복은 무엇인가요?": ["한복"],
    "한국에서 가장 높은 산은 무엇인가요?": ["백두산", "한라산"],
    "한국의 국보 1호는 무엇인가요?": ["숭례문", "남대문"]
}

# 모델 응답 수집 함수
def collect_model_responses(model_func, questions):
    responses = {}
    for question in questions:
        response = model_func(question)
        responses[question] = response
    return responses

# 모델별 응답 수집
clova_responses = {}  # ask_clova_stream 함수로 수집한 예정


In [17]:
# 응답 수집 여서
clova_responses = collect_model_responses(ask_clova_stream, korean_qa.keys())
clova_responses

assistant
대한민국의 수도는 서울특별시장입니다.assistant
한국에서 가장 높은 산은 한라산입니다. 한라산의 정상인 백록담의 높이는 약 1,947미터로, 대한민국에서 가장 높은 지점으로 알려져 있습니다.assistant
세종대왕assistant
한국의 국화는 **무궁화**입니다. 무궁화는 오랜 역사를 가진 꽃으로, 한국의 자연환경에 잘 적응하며 아름다운 꽃을 피웁니다. 이 꽃은 한국인의 정신을 상징하며, 오랜 세월 동안 국민적 자긍심의 상징으로 여겨져 왔습니다. 무궁화는 그 아름다움과 함께 강인한 생명력을 지니고 있어, 한국인의 끈기와 지혜를 나타내는 중요한 상징으로 자리 잡고 있습니다.assistant
한라도는 제주도를 의미합니다. 제주도는 대한민국의 남쪽에 위치한 섬으로, 한라산이라는 높은 산이 중심에 자리잡고 있습니다.assistant
한국의 전통 집 양식은 한옥입니다. 한옥은 한국의 전통적인 주거 건축 양식으로, 자연과 조화를 이루며 계절 변화에 맞춰 생활하는 지혜를 담고 있습니다. 한옥의 주요 특징으로는 온돌(바닥 난방), 한식 지붕(기와나 나무로 만든 경사진 지붕), 그리고 자연 재료를 사용한 구조 등이 있습니다. 이러한 양식은 한국의 기후와 생활 방식에 최적화되어 있으며, 공간 활용과 자연 친화적인 설계로 유명합니다.assistant
삼일절은 1919년 3월 1일에 일어난 3.1 운동을 기념하는 날입니다. 3.1 운동은 일제 강점기 하에서 한국인들이 독립을 선언하고, 이를 알리기 위해 전국적으로 벌인 대규모 비폭력 시위입니다. 이 운동은 한국 독립운동의 중요한 전환점으로 여겨지며, 한국인의 자주독립 의지를 전 세계에 알린 사건으로 기록됩니다.assistant
한국의 전통 의복은 한복입니다. 한복은 한국의 고유한 의상으로, 특별한 날이나 명절, 결혼식 등 다양한 행사에서 착용됩니다. 한복은 여러 가지 색상과 디자인으로 만들어지며, 계절에 따라 다른 스타일의 한복을 선택하기도 합니다. 예를 들어, 봄과 가을에는 가벼운 소재의 한복을, 겨울에

{'한국의 수도는 어디인가요?': 'assistant\n대한민국의 수도는 서울특별시장입니다.',
 '한국에 제일높은 무엇인가요?': 'assistant\n한국에서 가장 높은 산은 한라산입니다. 한라산의 정상인 백록담의 높이는 약 1,947미터로, 대한민국에서 가장 높은 지점으로 알려져 있습니다.',
 '한글을 창제한 왕은 누구인가요?': 'assistant\n세종대왕',
 '한국의 국화는 무엇인가요?': 'assistant\n한국의 국화는 **무궁화**입니다. 무궁화는 오랜 역사를 가진 꽃으로, 한국의 자연환경에 잘 적응하며 아름다운 꽃을 피웁니다. 이 꽃은 한국인의 정신을 상징하며, 오랜 세월 동안 국민적 자긍심의 상징으로 여겨져 왔습니다. 무궁화는 그 아름다움과 함께 강인한 생명력을 지니고 있어, 한국인의 끈기와 지혜를 나타내는 중요한 상징으로 자리 잡고 있습니다.',
 '한라도는 어느 나라의 무엇인가요?': 'assistant\n한라도는 제주도를 의미합니다. 제주도는 대한민국의 남쪽에 위치한 섬으로, 한라산이라는 높은 산이 중심에 자리잡고 있습니다.',
 '한국의 전통 집 양식은 무엇이라고 하나요?': 'assistant\n한국의 전통 집 양식은 한옥입니다. 한옥은 한국의 전통적인 주거 건축 양식으로, 자연과 조화를 이루며 계절 변화에 맞춰 생활하는 지혜를 담고 있습니다. 한옥의 주요 특징으로는 온돌(바닥 난방), 한식 지붕(기와나 나무로 만든 경사진 지붕), 그리고 자연 재료를 사용한 구조 등이 있습니다. 이러한 양식은 한국의 기후와 생활 방식에 최적화되어 있으며, 공간 활용과 자연 친화적인 설계로 유명합니다.',
 '삼일절은 무엇을 기념하는 날인가요?': 'assistant\n삼일절은 1919년 3월 1일에 일어난 3.1 운동을 기념하는 날입니다. 3.1 운동은 일제 강점기 하에서 한국인들이 독립을 선언하고, 이를 알리기 위해 전국적으로 벌인 대규모 비폭력 시위입니다. 이 운동은 한국 독립운동의 중요한 전환점으로 여겨지며, 한국인의 자주독립 의지를 전

# 2. gemma3 4b

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저 로드 (함수 외부에서 한 번만 실행)
model_id = "google/gemma-3-4b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

def ask_gemma(question):
    # Gemma-3 대화 형식에 맞게 프롬프트 구성
    prompt = f"<start_of_turn>system\n 주어진 질문에 단답형으로 대답하세요.\n<end_of_turn>\n<start_of_turn>user\n{question}\n<end_of_turn>\n<start_of_turn>model"

    # 토크나이저를 사용하여 프롬프트 처리
    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to(model.device)

    # 추론 설정
    generation_config = {
        "max_new_tokens": 200,
        "do_sample": True,
        "temperature": 0.7,
        "top_p": 0.9,
    }

    # 모델을 통한 추론
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            **generation_config
        )

    # 입력 토큰 길이를 확인하여 모델의 응답만 추출
    response = tokenizer.decode(output_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

    # 응답에서 가능한 남은 시스템 태그 제거
    if "<end_of_turn>" in response:
        response = response.split("<end_of_turn>")[0].strip()

    return response



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
answer = ask_gemma("2.12와 2.9 둘 중 어떤 수가 더 클까?")
print(answer)


2.9가 더 큽니다.


In [ ]:
answer = ask_gemma("**코너**는 **토푸리아**보다 키가 크고 **토푸리아**는 **메랍**보다 키가 크다. 3명 중 키가 가장 큰 사람은 누구인가?")
print(answer)


코너


In [ ]:
korean_qa = {
    "한국의 수도는 어디인가요?": ["서울", "서울특별시"],
    "한국에 제일높은 무엇인가요?": ["백두산", "고조봉"],
    "한글을 창제한 왕은 누구인가요?": ["세종대왕", "세종", "이도"],
    "한국의 국화는 무엇인가요?": ["무궁화"],
    "한라도는 어느 나라의 무엇인가요?": ["한국", "대한민국"],
    "한국의 전통 집 양식은 무엇이라고 하나요?": ["한옥"],
    "삼일절은 무엇을 기념하는 날인가요?": ["3.1 운동", "삼일 운동", "독립 만세 운동"],
    "한국의 전통 의복은 무엇인가요?": ["한복"],
    "한국에서 가장 높은 산은 무엇인가요?": ["백두산", "한라산"],
    "한국의 국보 1호는 무엇인가요?": ["숭례문", "남대문"]
}

# 모델 응답 수집 함수
def collect_model_responses(model_func, questions):
    responses = {}
    for question in questions:
        print(f"질문: {question}")
        response = model_func(question)
        print(f"응답: {response}\n")
        responses[question] = response
    return responses

# Gemma 모델 응답 수집
gemma_responses = collect_model_responses(ask_gemma, korean_qa.keys())
gemma_responses

질문: 한국의 수도는 어디인가요?
응답: 
서울입니다.

질문: 한국에 제일높은 무엇인가요?
응답: 
제주도 한라산

질문: 한글을 창제한 왕은 누구인가요?
응답: 
세종대왕

질문: 한국의 국화는 무엇인가요?
응답: 
무궁화

질문: 한라도는 어느 나라의 무엇인가요?
응답: 
한라도 한국의 도시입니다.

질문: 한국의 전통 집 양식은 무엇이라고 하나요?
응답: 
한옥

질문: 삼일절은 무엇을 기념하는 날인가요?
응답: 
1919년 3월 1일, 대한민국 임시정부의 대한 독립 만세 운동이 일어난 것을 기념하는 날입니다.

질문: 한국의 전통 의복은 무엇인가요?
응답: 
한복

질문: 한국에서 가장 높은 산은 무엇인가요?
응답: 
한라산입니다.

질문: 한국의 국보 1호는 무엇인가요?
응답: 
석굴암 불상



{'한국의 수도는 어디인가요?': '\n서울입니다.',
 '한국에 제일높은 무엇인가요?': '\n제주도 한라산',
 '한글을 창제한 왕은 누구인가요?': '\n세종대왕',
 '한국의 국화는 무엇인가요?': '\n무궁화',
 '한라도는 어느 나라의 무엇인가요?': '\n한라도 한국의 도시입니다.',
 '한국의 전통 집 양식은 무엇이라고 하나요?': '\n한옥',
 '삼일절은 무엇을 기념하는 날인가요?': '\n1919년 3월 1일, 대한민국 임시정부의 대한 독립 만세 운동이 일어난 것을 기념하는 날입니다.',
 '한국의 전통 의복은 무엇인가요?': '\n한복',
 '한국에서 가장 높은 산은 무엇인가요?': '\n한라산입니다.',
 '한국의 국보 1호는 무엇인가요?': '\n석굴암 불상'}

#  rag

In [ ]:
!pip install -q langchain pypdf sentence-transformers chromadb openai langchain_community datasets

In [10]:
pth='/content/drive/MyDrive/프로젝트/[llm]기타실습/'

In [11]:
# Image 1
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from uuid import uuid4
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

loader = PyPDFLoader(
    file_path=pth  + "250122청년고용촉진특별위원회안건.pdf",
)
docs = loader.load()

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embeddings = OllamaEmbeddings(model="bge-m3")

client = QdrantClient(path="/tmp/0429/test_5")

# 기존 벡터DB 컬렉션 생성
client.create_collection(
    collection_name="kistep_0429_5",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

# Image 2
qdrant = QdrantVectorStore(
    client=client,
    collection_name="kistep_0429_5",
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
)

uuids = [str(uuid4()) for _ in range(len(splits))]

qdrant.add_documents(documents=splits, ids=uuids)

# 벡터 저장소를 검색기로 설정
retriever = qdrant.as_retriever()

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Image 3
# Define application steps
def retrieve(state: State):
    retrieved_docs = retriever.invoke(state["question"])
    return {"context": retrieved_docs}

def clova_generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    prompt_text = f"""
당신은 Q&A 전문 AI 어시스턴트입니다. 주어진 컨텍스트를 사용하여 질문에 답변해주세요.

컨텍스트:
{docs_content}

질문:
{state["question"]}

답변:
"""
    response = ask_clova_stream(prompt_text)
    return {"answer": response}

In [ ]:
# Compile application and test
clova_graph_builder = StateGraph(State).add_sequence([retrieve, clova_generate])
clova_graph_builder.add_edge(START, "retrieve")
clova_graph = clova_graph_builder.compile()


from IPython.display import Image, display

display(Image(clova_graph.get_graph().draw_mermaid_png()))


In [ ]:
for message, metadata in clova_graph.stream(
    {"question": "트럼프 정부 정책 변화에 따른 국가전략기술 대응 정책을 설명해주세요"}, stream_mode="messages"
):
    print(message.content, end="")
# 3.3s